In [1]:
from skimage import data, transform, io
from skimage.color import rgb2gray
import numpy as np
import pandas as pd
import napari
from napari.utils.transforms import Affine, CompositeAffine
import matplotlib.pyplot as plt
from MILWRM.MxIF import *

---
# Transform all marker images into ST space and save to TIFFs

In [2]:
def transform_tiffs(
    affine,
    shape,
    tiffdir,
    read_suffix="_stitched_downsample8.tif",
    save_suffix="_stitched_downsample8_6723_4.tif",
    read_mask="WD86055_TISSUE_MASK_stitched_downsample8.tif",
    save_mask="WD86055_TISSUE_MASK_stitched_downsampled8_6723_4.tif",
    flip=False,
):
    # individual markers first
    for marker in eliotmarkers:
        print(marker, end=": ")
        tmp = io.imread("{}/{}{}".format(tiffdir, marker, read_suffix))
        if flip:
            # flip horizontally the MxIF image
            tmp = tmp[::-1,::-1]
        image = transform.warp(tmp, transform.AffineTransform(affine), output_shape = abs(shape))
        # downsample image
        #image = block_reduce(image, block_size=(8, 8), func=np.mean, cval=0)
        print(image.shape)
        io.imsave(arr=image, fname="{}/{}{}".format(tiffdir, marker, save_suffix))
    # tissue mask if needed
    if read_mask is not None and save_mask is not None:
        print("tissue mask", end=": ")
        tmp = io.imread("{}/{}".format(tiffdir, read_mask))
        if flip:
            # flip horizontally the MxIF image
            tmp = tmp[::-1,::-1]
        image = transform.warp(tmp, transform.AffineTransform(affine), output_shape = abs(shape))
        print(image.shape)
        io.imsave(arr=image, fname="{}/{}".format(tiffdir, save_mask))

In [11]:
def transform_imgs(
    affine,
    shape,
    img_in,
    img_out,
    flip=False,
    downsample_fact=None,
    plot=True,
):
    a = img.from_npz(img_in)
    shape[2] = a.n_ch  # replace third dim of shape with number of channels in img
    if flip:
        a.img = transform.warp(a.img[::-1,::-1,:], transform.AffineTransform(affine), output_shape=shape)
        a.mask = transform.warp(a.mask[::-1,::-1], transform.AffineTransform(affine), output_shape=shape[:2])
    else:
        a.img = transform.warp(a.img, transform.AffineTransform(affine), output_shape=shape)
        a.mask = transform.warp(a.mask, transform.AffineTransform(affine), output_shape=shape[:2])
    if downsample_fact is not None:
        a.downsample(fact=downsample_fact)
        
    if plot:
        rgb = a.show(channels=["SMA","VIMENTIN","BCATENIN"], RGB=True)
    a.to_npz(img_out)

In [4]:
sample_key = pd.read_csv("../../../resources/visium_sample_key.csv", index_col=0)

In [ ]:
for s in ["8899_1_SG00001","8899_2_SG00001","8899_3_SG00002","8899_4_SG00002","8899_5_SG00003","8899_6_SG00003","8899_7_SG00004","8899_8_SG00004"]:
    block = sample_key.loc[s, "block_name"]
    visium_ID = sample_key.loc[s, "sample_key_short"]
    ws_flip = sample_key.loc[s, "ws_flip"]
    ws_flip = True if ws_flip == "T" else False
    
    # get affine files
    affine = np.genfromtxt("/mnt/500gb_ssd/home/cody/images/MxIF/{}/{}_downsample8_{}_affine.csv".format(block, block, visium_ID), delimiter=",")
    shape = np.genfromtxt("/mnt/500gb_ssd/home/cody/images/MxIF/{}/{}_downsample8_{}_shape.csv".format(block, block, visium_ID), delimiter=",")
    
    print("Transforming {}".format(s))
    transform_imgs(
        affine=affine,
        shape=shape,
        img_in=sample_key.loc[s, "ws_npz"],
        img_out=sample_key.loc[s, "MxIF_npz"],
        flip=ws_flip,
        downsample_fact=None,
        plot=True,
    )

Transforming 8899_1_SG00001
Loading img object from /mnt/500gb_ssd/home/cody/images/MxIF/SG00001/SG00001_downsample8.npz...
